# Guardrails

The ultimate goal of machine learning is to make accurate predictions on unseen data. One of the benefits of using EvalML to build models is that provides guardrails to ensure you are building pipelines that will perform reliably in the future.


The guardrails described here help you avoid overfitting to your data and ultimately help you build a model that will perform as you expect once it is deployed in to the real world.


## Cross-validation for pipeline evaluation

By default, EvalML performs 3-fold cross validation when building pipelines. This means that it evaluates each pipeline 3 times using different for training and testing. In each trial the data used for testing is has no overlap from the data used for training to avoid overfitting.

While this is a good baseline approach, you can pass your own cross validation object to be used during modeling. The cross validation object can be any of the CV methods defined in [scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html) or use a compatible API.

For example, if we wanted to do a time series split:


In [1]:
import evalml

from sklearn.model_selection import TimeSeriesSplit

X, y = evalml.demos.load_breast_cancer()

clf = evalml.AutoClassifier(
    cv=TimeSeriesSplit(n_splits=6), 
    max_pipelines=1
)

clf.fit(X, y)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Precision. Greater score is better.

Searching up to 1 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: linear_model, random_forest, xgboost

Testing LogisticRegression w/ imputation + scaling: 100%|██████████| 1/1 [00:02<00:00,  2.70s/it]

✔ Optimization finished


if we describe the 1 pipeline we built, we can see the scores for each of the 6 splits as determined by the cross-validation object we provided

In [2]:
clf.describe_pipeline(0)

************************
* Pipeline Description *
************************

Pipeline Name: LogisticRegression w/ imputation + scaling
Model type: linear_model
Objective: Precision (greater is better)
Total training time (including CV): 2.7 seconds

Parameters
• penalty: l2
• C: 8.444214828324364
• impute_strategy: most_frequent

Cross Validation
               F1  Precision  Recall   AUC  Log Loss  # Training  # Testing
0           0.942      0.942   0.942 0.973     0.195      83.000         81
1           0.964      1.000   0.964 0.994     0.103     164.000         81
2           0.981      0.981   0.981 0.988     0.147     245.000         81
3           0.973      1.000   0.973 1.000     0.048     326.000         81
4           0.984      1.000   0.984 0.998     0.099     407.000         81
5           0.992      1.000   0.992 1.000     0.042     488.000         81
mean        0.973      0.987   0.973 0.992     0.106     285.500         81
std         0.018      0.023   0.018 0.010  

## Detects unstable pipelines

When we perform cross validation we are trying generate an estimate of pipeline performance. EvalML does this by taking the mean of the score across the folds. If the performance across the folds varies greatly, it is indicative the the estimated value may be unreliable. 

To protect the user against this, EvalML check to see if performance of the pipeline has a high variance between different folds. It triggers a warning is the "coeffient of variance" of the scores (the standard deviation divided by mean) or the pipelines scores exeeds .2.

This warning will appear in the pipeline rankings under `high_variance_cv`.

In [5]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,0,LogisticRegressionPipeline,0.987298,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
